## Processing RAW Data

See notes in retrieve_data

Next set in mnt_point2 

Also start copying into /scratch/midway2/fsemler/out_dat using the admix_env.sh again

When its done rsync into mnt_point making sure everything else is cleared

Also update text in retrieve_data

In [4]:
import sys, os, json
sys.path.append('/Code/')
os.environ['XENON_CONFIG'] = '/Code/xenon.config'

In [5]:
import straxen
import cutax

2024-10-09 14:40:36,300 - admix - WARNING - Initializing utilix DB failed. You cannot do database operations


DB initialization failed


In [28]:
in_dir  = "/Code/mnt_point2"
out_dir = "/Code/strax_data2"

max_workers = 25

# Set up strax context
_database_init = False
we_are_the_daq = False
output_folder = out_dir
include_rucio_remote = False
include_online_monitor = False
include_rucio_local = False
download_heavy = False
_auto_append_rucio_local = False
# Idk which of those is the correct one but it works
_rucio_path = in_dir
_rucio_local_path = in_dir
_raw_paths = in_dir
_processed_paths = [in_dir, out_dir]

### Online Modified for local
st = cutax.contexts.xenonnt_online(   _database_init              = _database_init,
                                        we_are_the_daq              = we_are_the_daq,
                                        output_folder               = output_folder,
                                        include_rucio_remote        = include_rucio_remote,
                                        include_online_monitor      = include_online_monitor,
                                        include_rucio_local         = include_rucio_local,
                                        download_heavy              = download_heavy,
                                        _auto_append_rucio_local    = _auto_append_rucio_local,
                                        _rucio_path                 = _rucio_path,
                                        _rucio_local_path           = _rucio_local_path,
                                        _raw_paths                  = _raw_paths,
                                        _processed_paths            = _processed_paths,)


In [22]:
run_ids = {}
for i in os.listdir(in_dir):
    split_str = i.split('-')
    if len(split_str) == 3:
        run_ids[split_str[0]] = 0
run_ids = run_ids.keys()
run_ids = list(run_ids)

In [33]:

# About 2h to collect 14 or so datasets on server
# Another 2h rsync
# Processing : 1h20m? 





# At max 400GB a day, 300GB on a good day

In [34]:
import h5py
save_path = "/Code/processed_data"

vaild_ids = []
for i in run_ids:
    try:
        if not os.path.isfile(os.path.join(save_path, i + '.hdf5')):
            pat = st.get_array(str(i), max_workers = max_workers, targets='event_area_per_channel')
            ei = st.get_array(str(i), targets='event_info')
            if len(ei) > 4:
                # Save as HDF5 as it takes very long to load local data
                with h5py.File(os.path.join(save_path, i + '.hdf5'), 'w') as f:
                    f.create_dataset('event_area_per_channel', data=pat)
                    f.create_dataset('event_info', data=ei)
                vaild_ids.append(i)
    except Exception as e:
        print("Failed on {}".format(i))
        print(e)

Failed on 064662
raw_records for 064662 not found in any storage, and your context specifies it cannot be created.


## Run ID checking and merging

ID checking is unsuccessfull 

Just make a HDF5 file (keeping a backup), with the patterns and positions for each ID

In [4]:
in_dir  = "/Code/mnt_point"
out_dir = "/Code/strax_data2"

max_workers = 25

# Set up strax context
_database_init = False
we_are_the_daq = False
# Differnt dir so we still ahve some usable data should something mess up bigtime
output_folder = out_dir
include_rucio_remote = False
include_online_monitor = False
include_rucio_local = False
download_heavy = False
_auto_append_rucio_local = False
# Idk which of those is the correct one but it works
_rucio_path = in_dir
_rucio_local_path = in_dir
_raw_paths = in_dir
_processed_paths = [in_dir, out_dir]

### Online Modified for local
st = straxen.contexts.xenonnt_online(   _database_init              = _database_init,
                                        we_are_the_daq              = we_are_the_daq,
                                        output_folder               = output_folder,
                                        include_rucio_remote        = include_rucio_remote,
                                        include_online_monitor      = include_online_monitor,
                                        include_rucio_local         = include_rucio_local,
                                        download_heavy              = download_heavy,
                                        _auto_append_rucio_local    = _auto_append_rucio_local,
                                        _rucio_path                 = _rucio_path,
                                        _rucio_local_path           = _rucio_local_path,
                                        _raw_paths                  = _raw_paths,
                                        _processed_paths            = _processed_paths,)


In [9]:
# Make list of datasets not yet processed
valid_ids = []
save_path = "/Code/processed_data"
all_ids = '/Code/runids.txt' # Contains all valid ids for the Xenon calibration data
todo = []
# Manually reject datasets
flagged = []

for i in os.listdir(save_path):
    valid_ids.append(i.split('.')[0])
all_ids_list = []
with open(all_ids, 'r') as file:
    for line in file:
        all_ids_list.append(line.strip())

for i in all_ids_list:
    if i not in valid_ids and i not in flagged:
        todo.append(i)

# Only need 14 - 15 per 100GB
for i in range(20):
    print(todo[i])

065091
065090
065089
065065
065062
065061
065060
065057
065056
065055
065052
065051
065050
065047
065046
065045
065042
065041
065040
065037


In [10]:
todo[0:20]

['065091',
 '065090',
 '065089',
 '065065',
 '065062',
 '065061',
 '065060',
 '065057',
 '065056',
 '065055',
 '065052',
 '065051',
 '065050',
 '065047',
 '065046',
 '065045',
 '065042',
 '065041',
 '065040',
 '065037']

In [11]:
for i in range(20, 40):
    print(todo[i])

065036
065035
065032
065031
065030
065024
065019
064771
064768
064767
064766
064763
064762
064761
064758
064757
064756
064753
064752
064751


In [12]:
todo[20:40]

['065036',
 '065035',
 '065032',
 '065031',
 '065030',
 '065024',
 '065019',
 '064771',
 '064768',
 '064767',
 '064766',
 '064763',
 '064762',
 '064761',
 '064758',
 '064757',
 '064756',
 '064753',
 '064752',
 '064751']

In [13]:
for i in range(40, 60):
    print(todo[i])

064748
064747
064745
064743
064742
064739
064738
064737
064734
064728
064723
064705
064673
064672
064669
064668
064667
064664
064663
064662


In [14]:
todo[40:60]

['064748',
 '064747',
 '064745',
 '064743',
 '064742',
 '064739',
 '064738',
 '064737',
 '064734',
 '064728',
 '064723',
 '064705',
 '064673',
 '064672',
 '064669',
 '064668',
 '064667',
 '064664',
 '064663',
 '064662']